In [1]:
import datetime
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
tweets = pd.read_csv("./data/tweets_all.csv", encoding="latin1")

In [3]:
min_lat = tweets.LATITUDE.min()
min_long = tweets.LONGITUDE.min()

lat_range = tweets.LATITUDE.max() - tweets.LATITUDE.min()
long_range = tweets.LONGITUDE.max() - tweets.LONGITUDE.min()

lat_bin_size = lat_range/9
long_bin_size= long_range/9

In [4]:
tweets["long_bin"] = tweets.LONGITUDE.apply(lambda long: (min_long + (((long - min_long)//long_bin_size)*long_bin_size)))
tweets["lat_bin"] = tweets.LATITUDE.apply(lambda lat: (min_lat + (((lat - min_lat)//lat_bin_size)*lat_bin_size)))

In [5]:
tweets["long_bin"] = tweets["long_bin"].apply(lambda l: str(l))
tweets["lat_bin"] = tweets["lat_bin"].apply(lambda l: str(l))

In [6]:
tweets["lat_long_bin"] = tweets["lat_bin"] + "," + tweets["long_bin"]

In [7]:
def parse_tweets(tweet):

    tweet = re.sub('@[^\s]+','',tweet).strip()
    tweet = re.sub('#[^\s]+','',tweet).strip()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet).strip()
    tweet = " ".join(re.findall("[a-zA-Z]+", tweet))
    
    return tweet

In [8]:
tweets["parsed_tweets"] = tweets.MESSAGETEXT.apply(lambda tweet: parse_tweets(tweet))

In [9]:
# need to get pos tag
class StemmerTokenizer(object):

    def __init__(self):
        self.stemmer = PorterStemmer()
        
    def __call__(self, doc):
        is_noun = lambda pos: pos[:2] == 'NN'
        doc = " ".join(re.findall("[a-zA-Z]+", doc))
        tokenized = nltk.word_tokenize(doc)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]

In [10]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,long_bin,lat_bin,lat_long_bin,parsed_tweets
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0,-0.3293425440788269,51.375291188557945,"51.375291188557945,-0.3293425440788269",Not yet got tics for Liverpool waiting till Fr...
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0,-0.2335423231124878,51.466681162516274,"51.466681162516274,-0.2335423231124878",So Many Accents
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0,-0.04194188117980957,51.329596201578774,"51.329596201578774,-0.04194188117980957",If it doesn t make you happy by December st le...
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0,-0.04194188117980957,51.512376149495445,"51.512376149495445,-0.04194188117980957",thanks and are you open tomorrow at all
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0,0.14965856075286865,51.55807113647461,"51.55807113647461,0.14965856075286865",but he is


In [11]:
idf_by_lat_long_bin = {}

for lat_long_bin in tweets.lat_long_bin.unique():
    clf = TfidfVectorizer(tokenizer=StemmerTokenizer(), min_df=1, max_df=0.7, ngram_range=(1, 1), stop_words=set(stopwords.words('english')))
    try:
        clf.fit(tweets[tweets["lat_long_bin"] == lat_long_bin]["parsed_tweets"])
        word2idf = dict(zip(clf.get_feature_names(), clf.idf_))
        sorted_idf = sorted(word2idf.items(), key=lambda x: x[1], reverse=True)
    except ValueError:
        sorted_idf = []

    idf_by_lat_long_bin[lat_long_bin] = {
        "lat_long_bin": lat_long_bin,
        "sorted_idf": sorted_idf,
        "location": tweets[tweets["lat_long_bin"] == lat_long_bin]["LOCATION"]
    }
    
    print("Completed", lat_long_bin)

print("Finished!")

Completed 51.375291188557945,-0.3293425440788269
Completed 51.466681162516274,-0.2335423231124878
Completed 51.329596201578774,-0.04194188117980957
Completed 51.512376149495445,-0.04194188117980957
Completed 51.55807113647461,0.14965856075286865
Completed 51.649461110432945,-0.3293425440788269
Completed 51.603766123453774,-0.425142765045166
Completed 51.466681162516274,-0.13774210214614868
Completed 51.512376149495445,-0.13774210214614868
Completed 51.603766123453774,0.14965856075286865
Completed 51.329596201578774,-0.13774210214614868
Completed 51.42098617553711,-0.2335423231124878
Completed 51.512376149495445,-0.2335423231124878
Completed 51.42098617553711,0.05385833978652954
Completed 51.42098617553711,-0.3293425440788269
Completed 51.512376149495445,-0.5209429860115051
Completed 51.603766123453774,-0.2335423231124878
Completed 51.42098617553711,-0.5209429860115051
Completed 51.42098617553711,-0.425142765045166
Completed 51.466681162516274,-0.3293425440788269
Completed 51.5123761494

## Extract Spatial Key Terms

In [14]:
tweets = pd.read_csv("./data/tweets_all.csv", encoding="latin1")

In [15]:
tweets["datetime"] = pd.to_datetime(tweets.MESSAGEDATE)

In [16]:
tweets["datetime_to_nearest_hour"] = tweets.datetime.apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))

In [17]:
tweets["parsed_tweets"] = tweets.MESSAGETEXT.apply(lambda tweet: parse_tweets(tweet))

In [18]:
idf_by_hourly_interval = {}

for hour_interval in tweets.datetime_to_nearest_hour.unique():
    clf = TfidfVectorizer(tokenizer=StemmerTokenizer(), min_df=5, max_df=0.7, ngram_range=(1, 1), stop_words=set(stopwords.words('english')))
    clf.fit(tweets[tweets["datetime_to_nearest_hour"] == hour_interval]["parsed_tweets"])
    word2idf = dict(zip(clf.get_feature_names(), clf.idf_))
    sorted_idf = sorted(word2idf.items(), key=lambda x: x[1], reverse=True)

    idf_by_hourly_interval[hour_interval] = {
        "hour_interval": hour_interval,
        "sorted_idf": sorted_idf
    }
    
    print("Completed", hour_interval)

print("Finished!")

Completed 2014-12-31T09:00:00.000000000
Completed 2014-12-31T10:00:00.000000000
Completed 2014-12-31T08:00:00.000000000
Completed 2014-12-31T06:00:00.000000000
Completed 2014-12-31T07:00:00.000000000
Completed 2014-12-31T04:00:00.000000000
Completed 2014-12-31T05:00:00.000000000
Completed 2014-12-31T14:00:00.000000000
Completed 2014-12-31T15:00:00.000000000
Completed 2014-12-31T11:00:00.000000000
Completed 2014-12-31T13:00:00.000000000
Completed 2014-12-31T12:00:00.000000000
Completed 2014-12-31T18:00:00.000000000
Completed 2014-12-31T19:00:00.000000000
Completed 2014-12-31T16:00:00.000000000
Completed 2014-12-31T17:00:00.000000000
Completed 2014-12-31T23:00:00.000000000
Completed 2015-01-01T00:00:00.000000000
Completed 2014-12-31T20:00:00.000000000
Completed 2014-12-31T22:00:00.000000000
Completed 2015-01-01T08:00:00.000000000
Completed 2015-01-01T09:00:00.000000000
Completed 2015-01-01T04:00:00.000000000
Completed 2015-01-01T05:00:00.000000000
Completed 2015-01-01T06:00:00.000000000


In [ ]:
# TODO
# space
# idf_by_lat_long_bin["51.466681162516274,-0.16721909321271455"]["sorted_idf"][:1000]
# idf_by_lat_long_bin["51.512376149495445,-0.16721909321271455"]["sorted_idf"][:1000]
# idf_by_lat_long_bin["51.466681162516274,-0.12300360661286575"]["sorted_idf"][:1000]
# idf_by_lat_long_bin["51.512376149495445,-0.12300360661286575"]["sorted_idf"][:1000]
# idf_by_lat_long_bin["51.512376149495445,-0.07878812001301694"]["sorted_idf"][:1000]

# time
# idf_by_hourly_interval[np.datetime64('2015-01-01T00:00:00.000000000')]["sorted_idf"][:1000]
# idf_by_hourly_interval[np.datetime64('2015-01-01T18:00:00.000000000')]["sorted_idf"][:1000]
# idf_by_hourly_interval[np.datetime64('2015-01-01T19:00:00.000000000')]["sorted_idf"][:1000]
# idf_by_hourly_interval[np.datetime64('2015-01-01T21:00:00.000000000')]["sorted_idf"][:1000]
# idf_by_hourly_interval[np.datetime64('2015-01-01T16:00:00.000000000')]["sorted_idf"][:1000]

In [ ]:
searchfor = ['foot', 'chelsea', "tottenham"]

tweets[tweets["parsed_tweets"].str.contains("|".join(searchfor))].groupby(['datetime_to_nearest_hour'])['id'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)

In [24]:
key_words = {}

def get_common_idf(location, time, total_count = 0):
    
    count = 0
    for idf_lat_long in idf_by_lat_long_bin[location]["sorted_idf"]:
        for idf_time in idf_by_hourly_interval[np.datetime64(time)]["sorted_idf"]:
            if (idf_lat_long[0] == idf_time[0]) and (idf_lat_long[1] > 9) or (idf_time[1] > 9) :
                count += 1
                print(idf_lat_long, idf_time)
                if idf_lat_long[0] in key_words:
                    key_words[idf_lat_long[0]] +=1
                else:
                    key_words[idf_lat_long[0]] = 1
    total_count += count
#     print(count)
    return total_count

In [25]:
len(list(idf_by_hourly_interval.keys()))

48

In [26]:
len(list(idf_by_lat_long_bin.keys()))

83

In [27]:
times = idf_by_hourly_interval.keys()
# times = ["2015-01-01T00:00:00.000000000", "2015-01-01T18:00:00.000000000", "2015-01-01T19:00:00.000000000", "2015-01-01T21:00:00.000000000", "2015-01-01T16:00:00.000000000"]
locations = idf_by_lat_long_bin.keys()
total_count = 0

for location in locations:
    for time in times:
        prev_count = total_count
        total_count = get_common_idf(location, time, total_count)
        if total_count > prev_count:
            print(location)
            print(time)
            print('--------------')

('freez', 9.188272370947859) ('freez', 6.672750365408472)
('gym', 9.188272370947859) ('gym', 6.385068292956691)
('wcw', 9.188272370947859) ('wcw', 6.518599685581214)
('xxxxx', 9.188272370947859) ('xxxxx', 6.672750365408472)
51.466681162516274,-0.2335423231124878
2014-12-31T09:00:00.000000000
--------------
('bastard', 9.188272370947859) ('bastard', 6.970411137910149)
('biggest', 9.188272370947859) ('biggest', 6.816260458082891)
('courtesi', 9.188272370947859) ('courtesi', 6.970411137910149)
('dj', 9.188272370947859) ('dj', 6.970411137910149)
('hughton', 9.188272370947859) ('hughton', 6.459585514144159)
('inde', 9.188272370947859) ('inde', 6.816260458082891)
('knighthood', 9.188272370947859) ('knighthood', 6.970411137910149)
('load', 9.188272370947859) ('load', 6.816260458082891)
('murder', 9.188272370947859) ('murder', 6.970411137910149)
('rais', 9.188272370947859) ('rais', 6.970411137910149)
('rate', 9.188272370947859) ('rate', 6.816260458082891)
('stock', 9.188272370947859) ('stock',

('women', 9.188272370947859) ('women', 7.2285110035911835)
('youth', 9.188272370947859) ('youth', 7.2285110035911835)
51.466681162516274,-0.2335423231124878
2014-12-31T17:00:00.000000000
--------------
('amen', 9.188272370947859) ('amen', 7.275075845189281)
('amor', 9.188272370947859) ('amor', 7.275075845189281)
('asleep', 9.188272370947859) ('asleep', 7.275075845189281)
('await', 9.188272370947859) ('await', 7.275075845189281)
('biggest', 9.188272370947859) ('biggest', 7.275075845189281)
('bow', 9.188272370947859) ('bow', 7.120925165362023)
('freddi', 9.188272370947859) ('freddi', 6.233621970361121)
('lambeth', 9.188272370947859) ('lambeth', 7.275075845189281)
('load', 9.188272370947859) ('load', 6.668940041618966)
('niall', 9.188272370947859) ('niall', 7.120925165362023)
('river', 9.188272370947859) ('river', 6.987393772737501)
('rubbish', 9.188272370947859) ('rubbish', 7.275075845189281)
('til', 9.188272370947859) ('til', 6.869610737081117)
('trust', 9.188272370947859) ('trust', 7.2

51.466681162516274,-0.2335423231124878
2015-01-01T13:00:00.000000000
--------------
('aux', 9.188272370947859) ('aux', 6.7120812774708964)
('eaten', 9.188272370947859) ('eaten', 6.7120812774708964)
('gym', 9.188272370947859) ('gym', 6.5579305976436375)
('llama', 9.188272370947859) ('llama', 6.7120812774708964)
('opinion', 9.188272370947859) ('opinion', 6.7120812774708964)
51.466681162516274,-0.2335423231124878
2015-01-01T10:00:00.000000000
--------------
('ell', 9.188272370947859) ('ell', 7.030284538957564)
('floripa', 9.188272370947859) ('floripa', 7.030284538957564)
('gym', 9.188272370947859) ('gym', 6.876133859130306)
('load', 9.188272370947859) ('load', 6.742602466505783)
('messi', 9.188272370947859) ('messi', 6.876133859130306)
('oprincesssparkleo', 9.188272370947859) ('oprincesssparkleo', 7.030284538957564)
('podolski', 9.188272370947859) ('podolski', 7.030284538957564)
('product', 9.188272370947859) ('product', 7.030284538957564)
('toast', 9.188272370947859) ('toast', 6.74260246

('midfield', 9.188272370947859) ('midfield', 6.8271340229118564)
('motiv', 9.188272370947859) ('motiv', 7.600323911145338)
('newcastl', 9.188272370947859) ('newcastl', 7.312641838693557)
('product', 9.188272370947859) ('product', 7.600323911145338)
('race', 9.188272370947859) ('race', 7.600323911145338)
('scream', 9.188272370947859) ('scream', 7.600323911145338)
('shock', 9.188272370947859) ('shock', 6.994188107575022)
('soo', 9.188272370947859) ('soo', 7.600323911145338)
('southampton', 9.188272370947859) ('southampton', 5.728521734243746)
('stock', 9.188272370947859) ('stock', 7.600323911145338)
('tactic', 9.188272370947859) ('tactic', 6.8271340229118564)
('ting', 9.188272370947859) ('ting', 7.44617323131808)
('ty', 9.188272370947859) ('ty', 7.312641838693557)
('winner', 9.188272370947859) ('winner', 7.194858803037174)
('wood', 9.188272370947859) ('wood', 7.600323911145338)
('yaya', 9.188272370947859) ('yaya', 6.619494658133612)
51.466681162516274,-0.2335423231124878
2015-01-01T16:00

('terri', 9.188272370947859) ('terri', 6.021085549971084)
('wind', 9.188272370947859) ('wind', 7.725833642209509)
('wit', 9.188272370947859) ('wit', 7.320368534101345)
51.466681162516274,-0.2335423231124878
2015-01-01T19:00:00.000000000
--------------
('biggest', 9.188272370947859) ('biggest', 6.983216338251486)
('chrissi', 9.188272370947859) ('chrissi', 7.137367018078744)
('flash', 9.188272370947859) ('flash', 6.983216338251486)
('gym', 9.188272370947859) ('gym', 7.137367018078744)
('hungri', 9.188272370947859) ('hungri', 7.137367018078744)
('ilysm', 9.188272370947859) ('ilysm', 5.884604049583377)
('inde', 9.188272370947859) ('inde', 7.137367018078744)
('load', 9.188272370947859) ('load', 6.983216338251486)
('mixtap', 9.188272370947859) ('mixtap', 5.318208574662575)
('rate', 9.188272370947859) ('rate', 7.137367018078744)
('shock', 9.188272370947859) ('shock', 7.137367018078744)
('tobi', 9.188272370947859) ('tobi', 7.137367018078744)
('trek', 9.188272370947859) ('trek', 6.2900691576915

...

('gmt', 9.120142581909091) ('gmt', 6.935649768001586)
('gorgeou', 9.120142581909091) ('gorgeou', 7.089800447828844)
('harrod', 9.120142581909091) ('harrod', 6.802118375377063)
('healthi', 9.120142581909091) ('healthi', 6.578974824062853)
('hungri', 9.120142581909091) ('hungri', 7.089800447828844)
('legend', 9.120142581909091) ('legend', 7.089800447828844)
('reunit', 9.120142581909091) ('reunit', 7.089800447828844)
('review', 9.120142581909091) ('review', 7.089800447828844)
('thame', 9.120142581909091) ('thame', 6.483664644258528)
('trafalgar', 9.120142581909091) ('trafalgar', 6.578974824062853)
('yea', 9.120142581909091) ('yea', 7.089800447828844)
('yeni', 9.120142581909091) ('yeni', 7.089800447828844)
51.512376149495445,-0.2335423231124878
2014-12-31T22:00:00.000000000
--------------
('gmt', 9.120142581909091) ('gmt', 5.366278277705742)
51.512376149495445,-0.2335423231124878
2015-01-01T08:00:00.000000000
--------------
('gmt', 9.120142581909091) ('gmt', 6.111987788356544)
('latest', 9

('ell', 9.120142581909091) ('ell', 7.030284538957564)
('gmt', 9.120142581909091) ('gmt', 6.876133859130306)
('hat', 9.120142581909091) ('hat', 6.876133859130306)
('healthi', 9.120142581909091) ('healthi', 6.742602466505783)
('lool', 9.120142581909091) ('lool', 7.030284538957564)
('messi', 9.120142581909091) ('messi', 6.876133859130306)
('piccadilli', 9.120142581909091) ('piccadilli', 6.876133859130306)
('thame', 9.120142581909091) ('thame', 6.624819430849399)
('toast', 9.120142581909091) ('toast', 6.742602466505783)
51.512376149495445,-0.2335423231124878
2015-01-01T11:00:00.000000000
--------------
('airport', 9.120142581909091) ('airport', 7.225877690680423)
('album', 9.120142581909091) ('album', 7.071727010853165)
('chicken', 9.120142581909091) ('chicken', 7.225877690680423)
('crouch', 9.120142581909091) ('crouch', 7.071727010853165)
('donat', 9.120142581909091) ('donat', 5.583649955423331)
('fella', 9.120142581909091) ('fella', 6.938195618228642)
('gmt', 9.120142581909091) ('gmt', 7

('conced', 9.120142581909091) ('conced', 7.44617323131808)
('cost', 9.120142581909091) ('cost', 7.600323911145338)
('crap', 9.120142581909091) ('crap', 7.44617323131808)
('creat', 9.120142581909091) ('creat', 7.312641838693557)
('decent', 9.120142581909091) ('decent', 7.089498287379348)
('defeat', 9.120142581909091) ('defeat', 7.194858803037174)
('equalis', 9.120142581909091) ('equalis', 6.994188107575022)
('everton', 9.120142581909091) ('everton', 6.8271340229118564)
('excus', 9.120142581909091) ('excus', 7.600323911145338)
('explain', 9.120142581909091) ('explain', 7.600323911145338)
('frank', 9.120142581909091) ('frank', 6.133986842351911)
('gmt', 9.120142581909091) ('gmt', 7.44617323131808)
('gooner', 9.120142581909091) ('gooner', 7.600323911145338)
('hall', 9.120142581909091) ('hall', 7.600323911145338)
('heathrow', 9.120142581909091) ('heathrow', 7.600323911145338)
('henri', 9.120142581909091) ('henri', 7.600323911145338)
('hungri', 9.120142581909091) ('hungri', 7.600323911145338

('comeback', 9.120142581909091) ('comeback', 7.571682962382251)
('conced', 9.120142581909091) ('conced', 6.878535781822306)
('consid', 9.120142581909091) ('consid', 7.119697838639194)
('costa', 9.120142581909091) ('costa', 7.4381515697577285)
('dart', 9.120142581909091) ('dart', 7.571682962382251)
('decent', 9.120142581909091) ('decent', 7.725833642209509)
('defeat', 9.120142581909091) ('defeat', 7.119697838639194)
('dem', 9.120142581909091) ('dem', 7.4381515697577285)
('esio', 9.120142581909091) ('esio', 7.4381515697577285)
('excus', 9.120142581909091) ('excus', 7.4381515697577285)
('fabrega', 9.120142581909091) ('fabrega', 6.6272213535414)
('fantasi', 9.120142581909091) ('fantasi', 7.320368534101345)
('fella', 9.120142581909091) ('fella', 7.571682962382251)
('frank', 9.120142581909091) ('frank', 7.571682962382251)
('gea', 9.120142581909091) ('gea', 7.725833642209509)
('gmt', 9.120142581909091) ('gmt', 7.725833642209509)
('gorgeou', 9.120142581909091) ('gorgeou', 7.725833642209509)
('

('twice', 9.120142581909091) ('twice', 7.565264970035361)
('vol', 9.120142581909091) ('vol', 7.565264970035361)
('yaya', 9.120142581909091) ('yaya', 6.584435717023635)
51.512376149495445,-0.2335423231124878
2015-01-01T22:00:00.000000000
--------------
('gmt', 9.120142581909091) ('gmt', 5.74493212836325)
51.512376149495445,-0.2335423231124878
2015-02-01T03:00:00.000000000
--------------
('gmt', 9.120142581909091) ('gmt', 6.159875980425959)
('godfath', 9.120142581909091) ('godfath', 6.314026660253217)
('hungri', 9.120142581909091) ('hungri', 6.314026660253217)
('lool', 9.120142581909091) ('lool', 6.026344587801436)
51.512376149495445,-0.2335423231124878
2015-02-01T02:00:00.000000000
--------------


In [ ]:
51.466681162516274,-0.2335423231124878
2014-12-31T09:00:00.000000000
('freez', 9.188272370947859) ('freez', 6.672750365408472)
('gym', 9.188272370947859) ('gym', 6.385068292956691)
('wcw', 9.188272370947859) ('wcw', 6.518599685581214)
('xxxxx', 9.188272370947859) ('xxxxx', 6.672750365408472)

51.466681162516274,-0.2335423231124878
2014-12-31T10:00:00.000000000
('bastard', 9.188272370947859) ('bastard', 6.970411137910149)
('biggest', 9.188272370947859) ('biggest', 6.816260458082891)
('courtesi', 9.188272370947859) ('courtesi', 6.970411137910149)
('dj', 9.188272370947859) ('dj', 6.970411137910149)
('hughton', 9.188272370947859) ('hughton', 6.459585514144159)
('inde', 9.188272370947859) ('inde', 6.816260458082891)
('knighthood', 9.188272370947859) ('knighthood', 6.970411137910149)
('load', 9.188272370947859) ('load', 6.816260458082891)
('murder', 9.188272370947859) ('murder', 6.970411137910149)
('rais', 9.188272370947859) ('rais', 6.970411137910149)
('rate', 9.188272370947859) ('rate', 6.816260458082891)
('stock', 9.188272370947859) ('stock', 6.970411137910149)
('trust', 9.188272370947859) ('trust', 6.682729065458369)
('tune', 9.188272370947859) ('tune', 6.970411137910149)
('women', 9.188272370947859) ('women', 6.054120406035994)

51.466681162516274,-0.2335423231124878
2014-12-31T07:00:00.000000000
('snow', 9.188272370947859) ('snow', 5.608497976080766)

51.466681162516274,-0.2335423231124878
2014-12-31T04:00:00.000000000
('denni', 9.188272370947859) ('denni', 4.998200701669198)

51.466681162516274,-0.2335423231124878
2014-12-31T05:00:00.000000000
('lewisham', 9.188272370947859) ('lewisham', 5.1404857182199555)

51.466681162516274,-0.2335423231124878
2014-12-31T14:00:00.000000000
('condemn', 9.188272370947859) ('condemn', 6.752925310315798)
('democraci', 9.188272370947859) ('democraci', 6.647564794657972)
('disabl', 9.188272370947859) ('disabl', 7.004239738596705)
('express', 9.188272370947859) ('express', 7.004239738596705)
('gym', 9.188272370947859) ('gym', 6.870708345972182)
('heel', 9.188272370947859) ('heel', 7.158390418423963)
('hopkin', 9.188272370947859) ('hopkin', 7.004239738596705)
('howev', 9.188272370947859) ('howev', 7.158390418423963)
('inde', 9.188272370947859) ('inde', 7.158390418423963)
('kati', 9.188272370947859) ('kati', 7.158390418423963)
('motiv', 9.188272370947859) ('motiv', 7.158390418423963)
('nigger', 9.188272370947859) ('nigger', 7.158390418423963)
('panto', 9.188272370947859) ('panto', 7.158390418423963)
('river', 9.188272370947859) ('river', 7.158390418423963)
('shoot', 9.188272370947859) ('shoot', 7.004239738596705)
('smith', 9.188272370947859) ('smith', 7.158390418423963)
('technic', 9.188272370947859) ('technic', 7.158390418423963)
('tribun', 9.188272370947859) ('tribun', 7.158390418423963)
('trust', 9.188272370947859) ('trust', 6.647564794657972)
('truth', 9.188272370947859) ('truth', 6.870708345972182)
('unearth', 9.188272370947859) ('unearth', 7.158390418423963)
('women', 9.188272370947859) ('women', 6.4652432378640174)
('wood', 9.188272370947859) ('wood', 7.158390418423963)

51.466681162516274,-0.2335423231124878
2014-12-31T15:00:00.000000000
('dj', 9.188272370947859) ('dj', 7.196104626857632)
('elect', 9.188272370947859) ('elect', 7.196104626857632)
('flipagram', 9.188272370947859) ('flipagram', 7.196104626857632)
('honest', 9.188272370947859) ('honest', 7.196104626857632)
('inde', 9.188272370947859) ('inde', 7.041953947030374)
('kati', 9.188272370947859) ('kati', 7.196104626857632)
('key', 9.188272370947859) ('key', 6.790639518749468)
('paddington', 9.188272370947859) ('paddington', 6.908422554405851)
('product', 9.188272370947859) ('product', 7.196104626857632)
('quo', 9.188272370947859) ('quo', 7.196104626857632)
('trust', 9.188272370947859) ('trust', 7.196104626857632)

In [150]:
total_count

13905

In [95]:
sorted(key_words.items(), key=lambda x: x[1], reverse=True)

[('abl', 60),
 ('account', 50),
 ('ah', 42),
 ('accept', 39),
 ('age', 28),
 ('acc', 24),
 ('ago', 20),
 ('agre', 20),
 ('adam', 20),
 ('af', 18),
 ('alcohol', 14),
 ('afternoon', 12),
 ('abus', 12),
 ('admit', 12),
 ('abi', 11),
 ('ahh', 9),
 ('abbey', 8),
 ('absolut', 8),
 ('ad', 8),
 ('alright', 8),
 ('around', 8),
 ('alway', 8),
 ('airport', 6),
 ('ac', 5),
 ('across', 5),
 ('achiev', 5),
 ('add', 5),
 ('along', 5),
 ('anyway', 5),
 ('anyth', 5),
 ('allow', 5),
 ('addict', 4),
 ('ador', 4),
 ('anoth', 4),
 ('album', 4),
 ('al', 4),
 ('american', 4),
 ('advic', 3),
 ('anyon', 3),
 ('address', 3),
 ('air', 3),
 ('amen', 3),
 ('alreadi', 3),
 ('aim', 2),
 ('agent', 2),
 ('away', 2),
 ('beauti', 2),
 ('ask', 2),
 ('bed', 2),
 ('allah', 2),
 ('ahead', 2),
 ('amount', 2),
 ('argument', 2),
 ('also', 2),
 ('care', 2),
 ('america', 1),
 ('ahhh', 1),
 ('aint', 1),
 ('andi', 1),
 ('arsen', 1),
 ('awak', 1),
 ('argu', 1),
 ('awkward', 1),
 ('best', 1),
 ('cold', 1),
 ('came', 1)]

In [34]:
# 51.512376149495445,0.24545878171920776
# 2014-12-31T08:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 6.255757752129241)

# 51.512376149495445,0.24545878171920776
# 2015-01-01T14:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 7.383225063715033)

# 51.512376149495445,0.24545878171920776
# 2015-01-01T13:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 7.348847573630395)

# 51.512376149495445,0.24545878171920776
# 2015-01-01T10:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 6.7120812774708964)

# 51.512376149495445,0.24545878171920776
# 2015-01-01T12:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 7.225877690680423)

# 51.512376149495445,0.24545878171920776
# 2015-01-01T17:00:00.000000000
# ('airport', 5.61512051684126) ('airport', 7.324358962381311)

In [55]:
tweets_at_coords = tweets[tweets["lat_long_bin"] == "51.512376149495445,0.24545878171920776"]

tweets_at_coords[tweets_at_coords.parsed_tweets.str.contains("airport")]

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,long_bin,lat_bin,lat_long_bin,parsed_tweets
16543,550358426539528194,NaN,0.254575,51.55541,0.254575,51.555408,31/12/2014 18:30:48,1563558049,upminster13,What's a dart players favorite airport Heathrow,Romford; London,NaN,0,0.24545878171920776,51.512376149495445,"51.512376149495445,0.24545878171920776",What s a dart players favorite airport Heathrow


In [57]:
tweets[tweets.parsed_tweets.str.contains("airport")]

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,long_bin,lat_bin,lat_long_bin,parsed_tweets
139,550215342677843968,NaN,-0.487881,51.471680,-0.487881,51.471680,31/12/2014 09:02:14,137175146,obaa_boni,Omg Omg Omg there's this fine as man in the ai...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",Omg Omg Omg there s this fine as man in the ai...
839,550224754838634496,NaN,-0.489448,51.472065,-0.489448,51.472065,31/12/2014 09:39:38,19987346,EmmaKParky,In the airport waiting for my flight and there...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",In the airport waiting for my flight and there...
2895,550230068807892992,NaN,-0.468273,51.471510,-0.468273,51.471512,31/12/2014 10:00:45,137175146,obaa_boni,I can't even buy food. Everything in the Lond...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",I can t even buy food Everything in the London...
3549,550179981163507715,NaN,-0.102889,51.544327,-0.102889,51.544327,31/12/2014 06:41:44,343197788,kickalert,@julesjoseph Fairport Convention at @unionchap...,Islington; London,NaN,0,-0.13774210214614868,51.512376149495445,"51.512376149495445,-0.13774210214614868",Fairport Convention at Mar London UK
3870,550184482389905408,NaN,-0.448606,51.469936,-0.448606,51.469936,31/12/2014 06:59:37,1238600882,hollyms6,@HeathrowAirport thanks for the best airport s...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",thanks for the best airport security experienc...
4471,550198835096862720,NaN,-0.325371,51.516876,-0.325371,51.516876,31/12/2014 07:56:39,351652356,FergusRose,Dreadful drop off experience at Luton airport ...,Ealing; London,NaN,0,-0.3293425440788269,51.512376149495445,"51.512376149495445,-0.3293425440788269",Dreadful drop off experience at Luton airport ...
5203,550186765940969473,NaN,-0.449158,51.471706,-0.449158,51.471706,31/12/2014 07:08:41,103015611,PRCAIngham,There's something rather unsettling about sitt...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",There s something rather unsettling about sitt...
5543,550206924768436225,NaN,-0.457358,51.470493,-0.457358,51.470493,31/12/2014 08:28:47,463712041,BasiC_Reedy,At le airport waiting for the goat @BasiC_Rate...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",At le airport waiting for the goat can t wait ...
5993,550296322532995072,NaN,-0.110786,51.462560,-0.110786,51.462559,31/12/2014 14:24:01,938939070,HomeAffairsPA,Woman; 26; from Haringey; London; arrested at ...,Lambeth; London,Syria,0,-0.13774210214614868,51.42098617553711,"51.42098617553711,-0.13774210214614868",Woman from Haringey London arrested at Luton a...
6773,550295283821993985,NaN,-0.437932,51.479786,-0.437932,51.479786,31/12/2014 14:19:54,72398127,JAZZYCAB,@Rayuny congratulations ! Spain . We in London...,Hillingdon; London,NaN,0,-0.5209429860115051,51.466681162516274,"51.466681162516274,-0.5209429860115051",congratulations Spain We in London need to fig...


In [124]:
for hourly_interval in idf_by_hourly_interval.keys():
    idf = idf_by_hourly_interval[hourly_interval]["sorted_idf"]
    if len([i for i in idf if ((i[1]>7.5))]) > 0:
        print(hourly_interval)
        print(len([i for i in idf if ((i[1]>7.5))])) 
        print("-------------------")

2015-01-01T00:00:00.000000000
262
-------------------
2015-01-01T18:00:00.000000000
245
-------------------
2015-01-01T16:00:00.000000000
175
-------------------
2015-01-01T21:00:00.000000000
243
-------------------
2015-01-01T20:00:00.000000000
149
-------------------
2015-01-01T19:00:00.000000000
279
-------------------
2015-01-01T22:00:00.000000000
158
-------------------


In [69]:
idf_by_hourly_interval[np.datetime64('2015-01-01T00:00:00.000000000')]["sorted_idf"][:1000]

[('ador', 7.809774870621934),
 ('alcohol', 7.809774870621934),
 ('apart', 7.809774870621934),
 ('appreci', 7.809774870621934),
 ('auguri', 7.809774870621934),
 ('awkward', 7.809774870621934),
 ('bath', 7.809774870621934),
 ('begin', 7.809774870621934),
 ('bell', 7.809774870621934),
 ('bon', 7.809774870621934),
 ('book', 7.809774870621934),
 ('bought', 7.809774870621934),
 ('british', 7.809774870621934),
 ('catch', 7.809774870621934),
 ('chanc', 7.809774870621934),
 ('class', 7.809774870621934),
 ('clear', 7.809774870621934),
 ('cross', 7.809774870621934),
 ('cup', 7.809774870621934),
 ('current', 7.809774870621934),
 ('david', 7.809774870621934),
 ('delight', 7.809774870621934),
 ('deserv', 7.809774870621934),
 ('di', 7.809774870621934),
 ('dinner', 7.809774870621934),
 ('ed', 7.809774870621934),
 ('either', 7.809774870621934),
 ('emot', 7.809774870621934),
 ('er', 7.809774870621934),
 ('es', 7.809774870621934),
 ('especi', 7.809774870621934),
 ('eu', 7.809774870621934),
 ('everywher',

In [113]:
for lat_long_bin in idf_by_lat_long_bin.keys():
    print(lat_long_bin)
    idf = idf_by_lat_long_bin[lat_long_bin]["sorted_idf"]
#     print([i for i in idf if ((i[1]>8) and (i[1]<9))])
    print(len([i for i in idf if ((i[1]>9))]))
    print("-------------------")

51.375291188557945,-0.3293425440788269
0
-------------------
51.466681162516274,-0.2335423231124878
4965
-------------------
51.329596201578774,-0.04194188117980957
0
-------------------
51.512376149495445,-0.04194188117980957
0
-------------------
51.55807113647461,0.14965856075286865
0
-------------------
51.649461110432945,-0.3293425440788269
0
-------------------
51.603766123453774,-0.425142765045166
0
-------------------
51.466681162516274,-0.13774210214614868
7544
-------------------
51.512376149495445,-0.13774210214614868
7342
-------------------
51.603766123453774,0.14965856075286865
0
-------------------
51.329596201578774,-0.13774210214614868
0
-------------------
51.42098617553711,-0.2335423231124878
0
-------------------
51.512376149495445,-0.2335423231124878
4747
-------------------
51.42098617553711,0.05385833978652954
0
-------------------
51.42098617553711,-0.3293425440788269
0
-------------------
51.512376149495445,-0.5209429860115051
0
-------------------
51.603766123

In [116]:
# Top temporal key words by IDF (where IDF > 9)
# 51.466681162516274,-0.2335423231124878 - Roehampton
# 51.466681162516274,-0.13774210214614868 - Clapham
# 51.512376149495445,-0.13774210214614868 - Soho
# 51.512376149495445,-0.2335423231124878 - White City

In [110]:
tweets[tweets["parsed_tweets"].str.contains("brunch")]

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,datetime,datetime_to_nearest_hour,parsed_tweets
4657,550203315926368256,NaN,-0.207275,51.477604,-0.207275,51.477604,31/12/2014 08:14:27,46715362,ChelseaBoy007,Feeling the cold? Why not enjoy brunch from 10...,Hammersmith; London,parsonsgreen,0,2014-12-31 08:14:27,2014-12-31 08:00:00,Feeling the cold Why not enjoy brunch from tod...
6146,550293222296403968,NaN,-0.128225,51.516870,-0.128225,51.516869,31/12/2014 14:11:42,559336525,AndreeaGB_,@b1305catalina brunch time #shoppingtime @ Joe...,Camden Town; London,shoppingtime,0,2014-12-31 14:11:42,2014-12-31 14:00:00,brunch time Joe the Juice New Oxford Street
6656,550302233418817538,NaN,-0.123593,51.512770,-0.123593,51.512772,31/12/2014 14:47:31,397526492,samrogers30,@KineticEcstasy having a well needed drink aft...,City of London; London,NaN,0,2014-12-31 14:47:31,2014-12-31 14:00:00,having a well needed drink after that brunch L...
7089,550304971309469697,NaN,-0.081878,51.527348,-0.081878,51.527348,31/12/2014 14:58:23,324909172,sarry_baby,NYE brunch with my ride or die @hannahapilates...,Hackney; London,NaN,0,2014-12-31 14:58:23,2014-12-31 14:00:00,NYE brunch with my ride or die Bill s Restaura...
7990,550294264887128065,NaN,-0.080705,51.419700,-0.080705,51.419701,31/12/2014 14:15:51,1354302756,JoannasRest,@SimplyJenkins No brunch I'm afraid. We open t...,Croydon; London,NaN,0,2014-12-31 14:15:51,2014-12-31 14:00:00,No brunch I m afraid We open tomorrow at midda...
10614,550247383922380801,NaN,-0.141250,51.492440,-0.141250,51.492439,31/12/2014 11:09:34,141422165,mac_kevin84,lapaq kronik !! jom brunch before explore city...,London; England,NaN,0,2014-12-31 11:09:34,2014-12-31 11:00:00,lapaq kronik jom brunch before explore city let s
11362,550255914318848002,NaN,-0.240930,51.346172,-0.240930,51.346172,31/12/2014 11:43:27,32602294,SassyMusacchio_,New Years Eve brunch,Epsom; England,NaN,0,2014-12-31 11:43:27,2014-12-31 11:00:00,New Years Eve brunch
11993,550239410156216320,NaN,-0.065230,51.498035,-0.065230,51.498035,31/12/2014 10:37:52,2932095629,see_us_hear_us,Time for a quick DiscoPump @TheGymGroup then b...,Camberwell; London,NaN,0,2014-12-31 10:37:52,2014-12-31 10:00:00,Time for a quick DiscoPump then brunch then re...
18690,550319522000166912,NaN,-0.085692,51.551760,-0.085692,51.551762,31/12/2014 15:56:13,19401472,YepStephenBrown,Boozy brunch and then a few in the pub. My res...,Islington; London,NaN,0,2014-12-31 15:56:13,2014-12-31 15:00:00,Boozy brunch and then a few in the pub My reso...
20379,550319122417197056,NaN,-0.207234,51.477562,-0.207234,51.477562,31/12/2014 15:54:37,46715362,ChelseaBoy007,Why not begin the year 2015 in style British b...,Hammersmith; London,parsonsgreen fulham,0,2014-12-31 15:54:37,2014-12-31 15:00:00,Why not begin the year in style British brunch...
